In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from pathlib import Path

# Configuración de la página
st.set_page_config(
    page_title="Mantenimiento Predictivo WAI4I 2020",
    layout="wide"
)

# Cargar datos
@st.cache_data
def load_data():
    # Construye la ruta al CSV desde el notebook
    notebook_path = Path().resolve()
    csv_path = notebook_path.parents[1] / "data" / "ai4i2020.csv"
    df = pd.read_csv(csv_path)
    return df

df = load_data()

# Columnas continuas y de fallos
numeric_cols = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", 
                "Torque [Nm]", "Tool wear [min]"]
fallos_cols = ["TWF","HDF","PWF","OSF","RNF"]

# Calcular conteo total de fallos
fallos_count = df[fallos_cols].sum()

# Sidebar para navegación
st.sidebar.title("Navegación")
opcion = st.sidebar.radio("Selecciona una sección:", ["Exploración de datos", "Predicciones"])

# Información de fallos
fault_info = pd.DataFrame({
    'Fault': ['Desgaste de herramienta ', 'Disipación de calor', 'Potencia fuera de rango', 
              'Sobreesfuerzo mecánico', 'Falla Aleatoria'],
    'Abbrev.': ['TWF', 'HDF', 'PWF', 'OSF', 'RNF']
})
st.sidebar.subheader("Significado de Fallos")
st.sidebar.table(fault_info)

# Contenido principal
st.title("Análisis y Predicción de Fallos")
st.markdown("Este panel permite explorar los datos del dataset WAI4I 2020 y analizar la distribución de fallos en máquinas industriales.")

# Exploración de datos
if opcion == "Exploración de datos":

    # Crear columnas para gráficos lado a lado
    col1, col2 = st.columns(2)

    # Pie chart de fallos con porcentajes
    with col1:
        st.subheader("Distribución de los tipos de fallos")
        counts = df[fallos_cols].sum()
        fallos_df = pd.DataFrame({
            "Tipo de fallo": counts.index,
            "Cantidad": counts.values,
            "Porcentaje": (counts.values / counts.sum() * 100).round(1)
        })
        fallos_df["label"] = fallos_df["Porcentaje"].astype(str) + "%"
        base = alt.Chart(fallos_df).encode(
            theta=alt.Theta(field="Cantidad", type="quantitative"),
            color=alt.Color(field="Tipo de fallo", type="nominal")
        )
        pie = base.mark_arc(innerRadius=0)
        text = base.mark_text(radius=100, size=14).encode(text="label")
        chart = (pie + text).properties(width=300, height=300)
        st.altair_chart(chart)

    # Gráfico interactivo: fallos por tipo de máquina
    with col2:
        st.subheader("Cantidad de fallos por tipo de máquina y tipo de fallo")
        df["MachineType"] = df["Product ID"].str[0]
        fail_long = (
            df.groupby(["MachineType"])[fallos_cols]
            .sum()
            .reset_index()
            .melt(id_vars="MachineType", var_name="Tipo de fallo", value_name="Cantidad")
        )
        stacked_bar = (
            alt.Chart(fail_long)
            .mark_bar()
            .encode(
                x=alt.X("Tipo de fallo:N", title="Tipo de fallo"),
                y=alt.Y("sum(Cantidad):Q", title="Cantidad total"),
                color=alt.Color("MachineType:N", title="Tipo de máquina"),
                tooltip=["Tipo de fallo:N", "MachineType:N", "Cantidad:Q"]
            )
            .properties(width=500, height=350)
        )
        st.altair_chart(stacked_bar, use_container_width=True)

    # Crear otra fila de gráficos
    col3, col4 = st.columns(2)

    # Gráfico de líneas: fallos según variable
    with col3:
        st.subheader("Cantidad de fallos según variable seleccionada")
        selected_var = st.selectbox("Selecciona la variable X:", numeric_cols)
        selected_fail = st.selectbox("Selecciona el tipo de fallo:", fallos_cols)
        num_bins = st.slider("Número de bins:", min_value=5, max_value=50, value=10)
        df_plot = df[[selected_var, selected_fail]].copy()
        df_plot["bin"] = pd.cut(df_plot[selected_var], bins=num_bins)
        fail_counts = df_plot.groupby("bin")[selected_fail].sum().reset_index()
        fail_counts["bin_str"] = fail_counts["bin"].astype(str)
        line_fail = alt.Chart(fail_counts).mark_line(point=True, color="salmon").encode(
            x=alt.X("bin_str:N", title=selected_var),
            y=alt.Y(f"{selected_fail}:Q", title=f"Cantidad de {selected_fail}"),
            tooltip=["bin_str", selected_fail]
        ).properties(width=500, height=300)
        st.altair_chart(line_fail)

    # Matriz de co-ocurrencia de fallos
    with col4:
        st.subheader("Matriz de co-ocurrencia de fallos")
        co_occur = df[fallos_cols].T.dot(df[fallos_cols])
        fig, ax = plt.subplots(figsize=(6,5))
        sns.heatmap(co_occur, annot=True, fmt="d", cmap="Blues", ax=ax)
        st.pyplot(fig)

#Predicciones
elif opcion == "Predicciones":
    st.header("📈 Predicciones")
    st.write("Aquí se implementará el modelo y su interfaz para hacer predicciones con nuevos datos.")


2025-12-04 17:58:35.931 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 17:58:35.934 No runtime found, using MemoryCacheStorageManager
2025-12-04 17:58:35.935 No runtime found, using MemoryCacheStorageManager
2025-12-04 17:58:35.935 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 17:58:35.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 17:58:35.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 17:58:35.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 17:58:35.971 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-04 17:58:35.972 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor